In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import ast
from textblob import TextBlob


In [71]:
archivo_games = "steam_games.json.gz"

# Lista para almacenar los datos JSON
data_games = []

# Abrir el archivo comprimido y leerlo línea por línea
with gzip.open("steam_games.json.gz", 'rt', encoding='utf-8') as f:
    for line in f:
        # Cargar cada línea como un objeto JSON y agregarlo a la lista
        data_games.append(json.loads(line))

# Convertir la lista de objetos JSON a un DataFrame de Pandas
df_games = pd.json_normalize(data_games)


In [81]:
# Ruta al archivo comprimido JSON
archivo_games = "steam_games.json.gz"

# Abre el archivo comprimido y carga los datos JSON
with gzip.open(archivo_games, 'rt', encoding='utf-8') as archivo_comprimido:
    # Carga los datos JSON en un DataFrame
    df_games = pd.read_json(archivo_comprimido, lines=True)

In [165]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [82]:
print(df_games.isnull().sum())

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64


In [83]:
df_games = df_games.dropna(how='all')
print(df_games.isnull().sum())

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


In [ ]:
df_games = df_games.drop(columns=['publisher', 'url','reviews_url','specs','early_access','app_name'])
# Limpiar la columna release_date y convertir todos los formatos a YYYY-MM-DD
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')
df_games = df_games.dropna(subset=['developer', 'price', 'tags','genres','release_date','id'])

print(df_games.isnull().sum())

In [94]:
df_games=df_games.reset_index()
df_games = df_games.drop(columns=['index'])
df_games['release_date'] = df_games['release_date'].dt.year

df_games['id'] = df_games['id'].astype(float)
df_games = df_games.rename(columns={'id': 'item_id'})

In [103]:
print(df_games.dtypes)

genres           object
title            object
release_date      int32
tags             object
price           float64
item_id         float64
developer        object
dtype: object


In [97]:
duplicados = df_games.duplicated(subset=['developer', 'price','release_date','item_id'])
#'tags','genres' exclui las lista por que generava error y no me parecia demaciado relevante 
 
print(f"Número de filas duplicadas: {(duplicados).sum()}")

Número de filas duplicadas: 0


In [91]:
df_games = df_games[~duplicados]

In [33]:

df_games.head()

,genres,title,release_date,tags,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd


In [98]:

valores_unicos = df_games['developer'].unique()
conteo_valores = df_games['developer'].value_counts()

# Mostrar el recuento de valores únicos
print("Recuento de valores únicos de la columna 'developer':")
print(conteo_valores)
num_valores_unicos = df_games['developer'].nunique()

df_games['developer'] = df_games['developer'].astype(str)
# Imprimir el número de valores únicos
print(f"Número de valores únicos en la columna 'developer': {num_valores_unicos}")



Recuento de valores únicos de la columna 'developer':
developer
Ubisoft - San Francisco       1258
SmiteWorks USA, LLC            812
Dovetail Games                 253
KOEI TECMO GAMES CO., LTD.     214
Paradox Development Studio     153
                              ... 
Project Purity Team              1
Triple rush games                1
Storybird,Julien Rocca           1
Light                            1
Bidoniera Games                  1
Name: count, Length: 10587, dtype: int64
Número de valores únicos en la columna 'developer': 10587


In [99]:
# Convertir la columna 'price' a numérico, los valores no convertibles serán NaN
df_games['price_numeric'] = pd.to_numeric(df_games['price'], errors='coerce')

# Filtrar los valores que no son numéricos
valores_no_numericos = df_games[df_games['price_numeric'].isna()]['price']

# Obtener el recuento de valores únicos no numéricos
conteo_valores_no_numericos = valores_no_numericos.value_counts()

# Mostrar el recuento de valores únicos no numéricos
print("Recuento de valores únicos no numéricos de la columna 'price':")
print(conteo_valores_no_numericos)

# Obtener el número de valores únicos no numéricos
num_valores_no_numericos_unicos = valores_no_numericos.nunique()

# Imprimir el número de valores únicos no numéricos
print(f"Número de valores únicos no numéricos en la columna 'price': {num_valores_no_numericos_unicos}")

df_games = df_games.drop(columns=['price_numeric'])

Recuento de valores únicos no numéricos de la columna 'price':
price
Free                             675
Free to Play                     468
Free To Play                     368
Free Mod                           4
Free Demo                          3
Play for Free!                     2
Third-party                        2
Play Now                           2
Install Now                        1
Play WARMACHINE: Tactics Demo      1
Install Theme                      1
Free HITMAN™ Holiday Pack          1
Play the Demo                      1
Free to Try                        1
Free to Use                        1
Name: count, dtype: int64
Número de valores únicos no numéricos en la columna 'price': 15


In [100]:
# Definir un diccionario de mapeo de valores a reemplazar y su valor de reemplazo
mapeo_reemplazo = {'Free': '0', 'Free To Play': '0', 'Free Mod' : '0' ,'Free Demo' : '0' ,'Play for Free!' :'0' ,'Third-party' : '0' ,'Play Now' : '0' ,'Install Now' : '0' ,'Play WARMACHINE: Tactics Demo' : '0' ,'Install Theme' : '0' ,'Free HITMAN™ Holiday Pack' : '0' ,'Play the Demo' : '0' ,'Free to Try' : '0' ,'Free to Use' : '0','Free to Play' : '0'}

# Reemplazar los valores en la columna 'price' según el diccionario de mapeo
df_games['price'] = df_games['price'].replace(mapeo_reemplazo)
df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce')
#esta ultima linia combierte los '0' = 0.00 

In [102]:
df_filtrado = df_games[df_games['price'] == 0.00]
valores_unicos = df_filtrado['developer'].unique()
conteo_valores = df_filtrado['developer'].value_counts()

# Mostrar el recuento de valores únicos
print("Recuento de valores únicos de la columna 'developer':")
print(conteo_valores)
num_valores_unicos = df_filtrado['developer'].nunique()

# Imprimir el número de valores únicos
print(f"Número de valores únicos en la columna 'developer': {num_valores_unicos}")
df_filtrado.head()

Recuento de valores únicos de la columna 'developer':
developer
Monster Games                 20
KOEI TECMO GAMES CO., LTD.    10
AMPLITUDE Studios              9
Quiet River                    9
Milestone S.r.l.               8
                              ..
Crankies Smartworks            1
Deck13                         1
YIMING ZHANG                   1
SandorHQ                       1
KUMA GAMES                     1
Name: count, Length: 1206, dtype: int64
Número de valores únicos en la columna 'developer': 1206


,genres,title,release_date,tags,price,item_id,developer
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.0,643980.0,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,"[Free to Play, Simulation, Sports, Casual, Ind...",0.0,670290.0,Poolians.com
39,"[Casual, Indie]",Zen of Sudoku,2006,"[Casual, Indie, Puzzle]",0.0,4900.0,Unknown Worlds Entertainment
162,[Action],Mare Nostrum,2008,"[Action, World War II, FPS, Mod, Multiplayer, ...",0.0,1230.0,Sandstorm Productions
217,"[Action, Simulation]",Darkest Hour: Europe '44-'45,2009,"[World War II, FPS, Action, Shooter, Simulatio...",0.0,1280.0,Darklight Games


In [104]:

# Ruta al archivo JSON original
#archivo_json = 'df_user__items.json'

# Cargar el archivo JSON en un DataFrame de pandas
#df = pd.read_json(archivo_json)

# Ruta donde se guardará el archivo Parquet comprimido
archivo_parquet = 'output_steam_games.parquet'

# Guardar el DataFrame como un archivo Parquet comprimido
df_games.to_parquet(archivo_parquet, compression='snappy')

print("Archivo JSON comprimido y guardado como Parquet.")


Archivo JSON comprimido y guardado como Parquet.


In [ ]:
# Leer el archivo Parquet y cargarlo en un DataFrame
df = pd.read_parquet('australian_users_items.parquet')

# Imprimir las primeras filas del DataFrame para verificar el contenido
df.head()

In [ ]:
# Leer el archivo Parquet y cargarlo en un DataFrame
df = pd.read_parquet('user_reviews.parquet')

# Imprimir las primeras filas del DataFrame para verificar el contenido
df.head()

In [4]:



# Leer el archivo Parquet y cargarlo en un DataFrame
df = pd.read_parquet('australian_users_items_comprimido.parquet')

# Imprimir las primeras filas del DataFrame para verificar el contenido
df.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [82]:
games = pd.read_parquet('output_steam_games.parquet')
games.head(5)

,genres,title,release_date,tags,price,item_id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980.0,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,"[Free to Play, Simulation, Sports, Casual, Ind...",0.00,670290.0,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd


In [84]:
games['tags'][4]

array(['Action', 'Adventure', 'Simulation', 'FPS', 'Shooter',
       'Third-Person Shooter', 'Sniper', 'Third Person'], dtype=object)

In [122]:
reviews = pd.read_parquet('user_reviews.parquet')
games = pd.read_parquet('output_steam_games.parquet')
df_filtrado = games[['item_id', 'developer']]

merged_df = pd.merge(reviews, df_filtrado, left_on='item_id', right_on='item_id')

merged_df.to_parquet('reviews_analysis.parquet', compression='snappy')



In [10]:
reviews = pd.read_parquet('user_reviews.parquet')
games = pd.read_parquet('output_steam_games.parquet')
df_filtrado = games[['item_id', 'developer','release_date']]
merged_df = pd.merge(reviews, df_filtrado, left_on='item_id', right_on='item_id')
merged_df = merged_df[(merged_df['recommend'] == True) & (merged_df['sentiment_analysis'] == 2)]
merged_df.to_parquet('best_developer_year.parquet', compression='snappy')
    

In [18]:
merged_df = pd.read_parquet('output_steam_games.parquet')
merged_df = merged_df[['release_date', 'price','developer']]
merged_df.to_parquet('developer.parquet', compression='snappy')

In [22]:

merged_df = pd.read_parquet(r'Dataset\best_developer_year.parquet')

In [23]:
merged_df.head()

,user_id,item_id,recommend,sentiment_analysis,developer,release_date
0,76561197970982479,1250.0,True,2,Tripwire Interactive,2009
1,76561197970982479,22200.0,True,2,ACE Team,2009
4,evcentric,248820.0,True,2,"Hopoo Games, LLC",2013
5,evcentric,370360.0,True,2,Zachtronics,2015
6,evcentric,237930.0,True,2,Supergiant Games,2014
